In [30]:
# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import joblib
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

In [32]:
# 2. LOAD DATA
df = pd.read_csv("uk_stations_clustered.csv")
print("Data loaded:", df.shape)

Data loaded: (10000, 23)


In [49]:
# 3. LOAD FEATURE LIST USED DURING TRAINING
with open("dnn_model_input_columns.json", "r") as f:
    feature_columns = json.load(f)

print("Model expects features:", feature_columns)
print("Number of features:", len(feature_columns))


Model expects features: ['operator', 'status', 'is_operational', 'latitude', 'longitude', 'num_points', 'connector_types', 'max_power_kw', 'borough', 'borough_density_km2', 'uk_avg_util_pct', 'uk_avg_energy_kWh', 'stations_per_borough', 'operational_flag', 'usage_Private - Restricted Access', 'usage_Privately Owned - Notice Required', 'usage_Public', 'usage_Public - Membership Required', 'usage_Public - Pay At Location', 'avg_power_per_connector', 'has_fast_charger', 'cluster']
Number of features: 22


In [53]:
# 4. PREPARE INPUT DATA (STRICT MATCH)
X_all = df[feature_columns].select_dtypes(include=[np.number])

print("Final prediction features:", X_all.columns.tolist())
print("Number of features:", X_all.shape[1])



# 5. LOAD SCALER AND TRANSFORM (IMPORTANT FIX)
scaler = joblib.load("priority_scaler.pkl")

# ✅ Convert to NumPy before scaling
X_all_scaled = scaler.transform(X_all.values)

Final prediction features: ['latitude', 'longitude', 'num_points', 'max_power_kw', 'borough_density_km2', 'uk_avg_util_pct', 'uk_avg_energy_kWh', 'stations_per_borough', 'operational_flag', 'avg_power_per_connector', 'has_fast_charger', 'cluster']
Number of features: 12


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [55]:
# 6. LOAD DNN MODEL
model = load_model(
    "dnn_priority_model_improved.h5",
    custom_objects={"mse": MeanSquaredError()}
)


In [57]:
# 7. PREDICT PRIORITY SCORE
df["predicted_priority"] = model.predict(X_all_scaled).flatten()

# 8. PRIORITY LABELS (DECISION LOGIC)
def priority_label(score):
    if score >= 0.15:
        return "High"
    elif score >= 0.08:
        return "Medium"
    else:
        return "Low"

df["priority_label"] = df["predicted_priority"].apply(priority_label)


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step


In [59]:
# 9. SAVE FINAL PRIORITY CSV
df.to_csv("ev_priority_predictions.csv", index=False)
print("Final priority CSV saved as ev_priority_predictions.csv")

Final priority CSV saved as ev_priority_predictions.csv


In [69]:
df.head()

,operator,status,is_operational,latitude,longitude,num_points,connector_types,max_power_kw,borough,borough_density_km2,...,usage_Private - Restricted Access,usage_Privately Owned - Notice Required,usage_Public,usage_Public - Membership Required,usage_Public - Pay At Location,avg_power_per_connector,has_fast_charger,cluster,predicted_priority,priority_label
0,Ev Dot,Not Operational,False,51.507351,-0.127758,1.0,Type 2 (Socket Only); Type 2 (Socket Only),7.0,Outer London,9318.948649,...,False,False,False,False,True,7.0,0,0,0.129647,Medium
1,Independent Operator,Operational,True,51.507291,-0.128896,1.0,0,4.8,Westminster,13608.400000,...,False,False,True,False,False,4.8,0,2,0.208053,High
2,Bp Pulse (Uk),Operational,True,51.507099,-0.130117,8.0,BS1363 3 Pin 13 Amp; Type 2 (Socket Only),7.0,Westminster,13608.400000,...,False,False,False,True,False,7.0,0,2,0.215987,High
3,Virta,Operational,True,51.508903,-0.125534,1.0,Type 2 (Socket Only),7.0,Westminster,9318.948649,...,False,False,False,False,False,7.0,0,1,0.131567,Medium
4,Bp Pulse (Uk),Operational,True,51.509162,-0.130650,4.0,Type 2 (Socket Only),4.0,Westminster,13608.400000,...,False,False,False,True,False,4.0,0,2,0.209763,High
